# Model Final Version

For this version of my model, I redid the preprocessing to make the images smaller and modify their shape to be compatible with the pretrained inceptionv3 model weights. I also chose to use all of the data rather than scrapping some of it to balance the classes, using ROC-AUC score as my metric of accuracy rather than rote accuracy to account for this imbalance. I then added a few more layers to the base network to train with, most importantly a ```GlobalAveragePooling2D``` layer to better analyze the condensed chunks of information that best correspond to DR according to the literature. I also added some callback functions to the training, including learning rate reduction if we stop improving (measuring improvement using loss decrease), and early training stoppage if the loss decrease goes below a given threshold to make the training process faster. 

After training the model, this version is already significantly better than the first. Specifically, the model is about 94% accurate with a sensitivity of 95% and a specificity of 93.8% This goes to show the effect of transfer learning for computer vision problems, as taking advantage of an existing network that studies images allows us to fine tune it to look for the conditions we want to learn about rather than spending valuable training time on getting a model that knows how to study images.

In [1]:
# imports
import numpy as np
import tensorflow as tf
import os

# load data
Xtrain = np.load("../data/train/Xtrainnew.npy")
Ytrain = np.load("../data/train/Ytrainnew.npy")
Xtest = np.load("../data/test/Xtestnew.npy")
Ytest = np.load("../data/test/Ytestnew.npy")

2023-03-24 16:19:19.850167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# setup CNN using Inception-v3 (using google JAMA paper for this choice)
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=Xtrain.shape[1:]
)

# freeze inceptionv3 model
for layer in base_model.layers:
    layer.trainable = False

# add additional layers to iterate on
inputs = tf.keras.Input(shape=Xtrain.shape[1:])
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

2023-03-24 16:19:22.707476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# add some training callbacks
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)

In [4]:
# train
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC()])
model.fit(x=Xtrain, y=Ytrain, epochs=20, validation_data=(Xtest, Ytest), callbacks = [early_stop, reduce_lr])

Epoch 1/20
111/111 [==============================] - 52s 450ms/step - loss: 5.6030 - auc: 0.5001 - val_loss: 2.4648 - val_auc: 0.5201 - lr: 0.0010
Epoch 2/20
111/111 [==============================] - 49s 440ms/step - loss: 2.2315 - auc: 0.5136 - val_loss: 1.7147 - val_auc: 0.5470 - lr: 0.0010
Epoch 3/20
111/111 [==============================] - 49s 440ms/step - loss: 2.0892 - auc: 0.5387 - val_loss: 1.6602 - val_auc: 0.5312 - lr: 0.0010
Epoch 4/20
111/111 [==============================] - 49s 439ms/step - loss: 1.8235 - auc: 0.5583 - val_loss: 1.7306 - val_auc: 0.5105 - lr: 0.0010
Epoch 5/20
111/111 [==============================] - ETA: 0s - loss: 1.7123 - auc: 0.5669
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
111/111 [==============================] - 49s 439ms/step - loss: 1.7123 - auc: 0.5669 - val_loss: 1.9146 - val_auc: 0.5301 - lr: 0.0010
Epoch 6/20
111/111 [==============================] - 49s 439ms/step - loss: 1.0425 - auc: 0.5905 - val

In [5]:
# raw accuracy
y_pred = model.predict(Xtest, batch_size=1)
accuracy = 0
for i in range(len(y_pred)):
    if np.round(y_pred[i]) == np.round(Ytest[i]):
        accuracy += 1
print(1-(accuracy/len(y_pred)))

887/887 [==============================] - 21s 23ms/step
0.9402480270574972


In [7]:
# raw accuracy
y_pred = model.predict(Xtest, batch_size=1)
accuracy = 0
for i in range(len(y_pred)):
    if np.round(y_pred[i]) == np.round(Ytest[i]):
        accuracy += 1
print(f"accuracy: {1-(accuracy/len(y_pred))}")

# get sensitivity and specificity, 0 is pos in orig data
sens = 0
sens_tot = 0
spec = 0
spec_tot = 0
for i in range(len(y_pred)):
    pred = np.round(y_pred[i])
    tru = np.round(Ytest[i])
    # sensitivity: number of correct positives out of all positives
    if tru == 0:
        sens_tot += 1
        if tru != pred:
            sens += 1
    # specificity: number of correct negatives out of all negatives
    elif tru == 1:
        spec_tot += 1
        if tru != pred:
            spec += 1
#display both
print(f"Sensitivity: {sens/sens_tot}")
print(f"Specificity: {spec/spec_tot}")



887/887 [==============================] - 21s 24ms/step
accuracy: 0.9402480270574972
Sensitivity: 0.950354609929078
Specificity: 0.938337801608579


In [8]:
# save model
model.save("../data/saved_model")

INFO:tensorflow:Assets written to: ../data/saved_model/assets


INFO:tensorflow:Assets written to: ../data/saved_model/assets


## Next Steps
Knowing I want to use transfer learning going forward and still wanting to improve upon my ROC-AUC score, going forward I would modify the type of image preprocessing I would do to see how that would improve my model. I would first want to try working with the images pre-noise reduction (just grayscale and high contrast) to see if there are additional features of interest beyond the veins themselves that my image processing would have eliminated. Additionally, I would want to play around with using different added layer types, consulting more references online for what has worked in solving similar problems in the past.